
# Отчет по тестовому заданию для стажёра в команду прикладных исследований



## Какая задача решалась

Для реализации была выбрана задача и метод решения этой задачи, описанные в статье «SPEAKER RECOGNITION FROM RAW WAVEFORM WITH SINCNET», Mirco Ravanelli, Yoshua Bengio, https://arxiv.org/pdf/1808.00158.pdf

Распознавание (классификация) говорящего - активная область исследований с приложениями в различных областях, таких как биометрическая аутентификация, криминалистика, безопасность, распознавание речи и отслеживание говорящего в аудиопотоке (speaker diarization - процесс разделения входящего аудиопотока на однородные сегменты в соответствии с идентификатором говорящего).


## В чём основная идея метода и в чём её отличие от других решений?

В статье предлагается новая архитектура CNN, названная SincNet, которая накладывает ограничения на первый сверточный слой для того, чтобы находить наиболее значимые фильтры. SincNet основан на параметризованных функциях sinc, которые реализуют полосовые фильтры. В отличие от стандартных CNN, которые оптимизируют все отсчеты каждого фильтра, только срезы низких и высоких частот напрямую извлекаются из данных с помощью предлагаемого метода. Это обеспечивает компактный и эффективный способ создания настраиваемого набора фильтров, специально для желаемого применения.
Идеальный полосовой фильтр (т. е. фильтр, где полоса пропускания идеально плоская, а затухание в полосе задерживания бесконечно) требует бесконечного числа отсчетов. Таким образом, любое усечение фильтра неизбежно приводит к отклонению от идеального фильтра, характеризуемого рябью в полосе пропускания и ограниченное затухание в полосе задерживания. Популярным решением этой проблемы является использование окон. Управление окнами выполняется путем умножения усеченной функции фильтра на оконную функцию w. В качестве функции w в статье использовалось окно Хэмминга.
Для тренировки сети использовался набор данных TIMIT —  (462 speakers, train chunk). Non-speech intervals at the beginning and end of each sentence were removed. To address text-independent speaker recognition, the calibration sentences of TIMIT (i.e., the utterances with the same text for all speakers) have been removed. Five sentences for each speaker were used for training, while the remaining three were used for test.


## Какой эксперимент ставился? Какие получились результаты и как их можно интерпретировать?

Сэмпл каждого речевого предложения был разделен на кадры по 200 мс (с перекрытием 10 мс), которые были переданы в архитектуру SincNet. Первый слой сети выполняет свертки на основе sinc, используя 80 фильтров длиной L = 251 отсчет. Затем в архитектуре используются два стандартных сверточных слоя, в каждом из которых используется 60 фильтров длиной 5. Нормализация слоев использовалась как для входных выборок, так и для всех сверточных слоев (включая входной уровень SincNet).
Затем были использованы три полносвязных слоя, состоящих из 2048 нейронов и нормализованных с помощью пакетной нормализации. Все скрытые слои используют функции активации ReLU. Параметры sinc-слоя были инициализированы с использованием частот отсечки мел-шкалы, в то время как остальная часть сети была инициализирована с помощью схемы инициализации «Glorot». Классификация спикеров на уровне кадра была получена путем применения классификатора softmax, обеспечивающего набор апостериорных вероятностей для целевых спикеров. Классификация на уровне предложения была получена путем усреднения прогнозов кадра и голосования за говорящего. 
В статье представлены результаты сравнения SincNet со стандартной CNN. Эти результаты, полученные на наборе данных TIMIT, показывают значение ошибки классификации Frame Error Rate (FER%) при использовании SincNet 33,0%, по сравнению с FER в 37,7%, достигнутым при базовой CNN.
Такие результаты можно объяснить следующим образом. Предлагаемый SincNet обладает рядом свойств:
• SincNet заставляет сеть сосредотачиваться только на параметрах фильтра, что существенно влияет на производительность. Предлагаемый подход использует информацию о форме фильтра, сохраняя при этом гибкость для адаптации к данным. Эти предварительные знания значительно упрощают изучение характеристик фильтра, помогая SincNet значительно быстрее сходиться к лучшему решению.
• SincNet резко уменьшает количество параметров в первом сверточном слое. Например, если мы рассмотрим слой, состоящий из F фильтров длины L, стандартная CNN использует параметры F · L по сравнению с 2F, рассматриваемым SincNet. Если F = 80 и L = 100, мы используем 8k параметров для CNN и только 160 для SincNet.
• Значения, полученные в первом сверточном слое, более интерпретируемы и удобочитаемы, чем при других подходах. Фактически, набор фильтров зависит только от параметров, имеющих ясный физический смысл.


## Как можно использовать полученный результат? Удалось ли приблизиться к цифрам из статьи? Какие есть перспективы для развития?

Натренированная сеть использовалась в статье для решения двух задач. 
а) идентификация спикера -  это совпадение 1:N, где голос сравнивается с несколькими шаблонами.
б) верификация спикера -  это совпадение 1:1, где голос одного говорящего сопоставляется с определенным шаблоном.
Для обеих задач  SincNet показал лучшие результаты по сравнению со стандартной CNN и другими методами не на основе нейронных сетей.
Мне удалось получить значение FER 49% после 75 эпох. Это немного уступает результату в статье (рис. 4) ~40%. Вероятно, это связано со случайной инициализацией весов сети.



На мой взгляд, предложенный в статье метод может быть также использован для решения задачи диаризации спикера — см. следующий пункт.


# Часть 2*

Мне показалась интересной статья «A Review of Speaker Diarization: Recent Advances with Deep Learning», Tae Jin Park, et al, https://arxiv.org/pdf/2101.09624.pdf

Задачу диаризации спикера можно рассмотреть для двух сценариев.

1. Для разговора двух человек. Необходимо отделить разговор от фонового шума. Привязать идентификатор к каждому спикеру. Затем система должна отслеживать спикеров до окончания сессии. 
 
2. Для множества людей. При использовании стереомикрофонов для каждого спикера рассчитывается направление. Затем это направление отслеживается.

Реализацию этих задач можно связать с содержанием предыдущего пункта. Тем более, что метод  SincNet реализован в, например, программном пакете  https://github.com/speechbrain/speechbrain. 

Этот пакет может быть использован для решения задачи диаризации. В частности, подходы https://speechbrain.github.io/tutorial_classification.html и  https://speechbrain.github.io/tutorial_separation.html
